In [1]:
import optuna

In [1]:
import imblearn
print(imblearn.__version__)

0.13.0


In [2]:
from imblearn.combine import SMOTETomek
print("Smotetomek succesfully imported") 

Smotetomek succesfully imported


In [3]:
import pandas as pd
df=pd.read_csv("social_demographic.csv")
df.head()

,age,gender,sex,education,Employment_status,Personal_Net_Income_Category,Ethnic_Background,living_arrangement,marital_status,hours_on_social_media,loneliness_score,social_media_frequency,mhi5_class_2022,gender.1,Religion,Religious_Membership,political_interest,mhi5_std_score_2022
0,60.0,999,1.0,6.0,1,7.0,2.0,3.0,1.0,2.0,0,1.0,0.0,999,NaN,2.0,1.0,84.0
1,32.0,999,2.0,4.0,1,4.0,2.0,3.0,1.0,18.0,0,3.0,1.0,999,10.0,1.0,2.0,52.0
2,49.0,999,1.0,4.0,1,4.0,1.0,2.0,1.0,17.0,0,6.0,0.0,999,NaN,2.0,2.0,64.0
3,70.0,999,1.0,6.0,9,6.0,1.0,2.0,1.0,0.0,0,5.0,0.0,999,NaN,2.0,1.0,76.0
4,60.0,999,1.0,4.0,4,5.0,1.0,3.0,1.0,6.0,0,7.0,0.0,999,NaN,2.0,1.0,88.0


In [8]:
import pandas as pd

# Define the target variable
target_column = ['mhi5_class_2022']

# Columns that are inside blocks but MAY only be needed for error analysis
possible_error_analysis_columns = ['gender', 'Religion', 'Religious_Membership', 'political_interest','mhi5_std_score_2022', 'gender.1', 'Ethnic_Background', 'social_media_frequency' ]

# Full list of extra columns
keep_columns = target_column + possible_error_analysis_columns 




# Separate features and target
X = df.drop(columns=keep_columns) 
y = df[target_column[0]]  # Target variable



In [10]:
X= X.reset_index(drop=True)
y= y.reset_index(drop=True)

In [12]:
print(X.shape)
print(y.shape)

(995, 9)
(995,)


In [14]:
print(X.isna().sum()) 
# the classidfer cannot work if some entries have NAN

age                             0
sex                             0
education                       2
Employment_status               0
Personal_Net_Income_Category    2
living_arrangement              0
marital_status                  0
hours_on_social_media           0
loneliness_score                0
dtype: int64


In [16]:
X.loc[:, "education"] = X["education"].fillna(X["education"].mean())
X.loc[:, "Personal_Net_Income_Category"] = X["Personal_Net_Income_Category"].fillna(X["Personal_Net_Income_Category"].mean())

In [18]:
print(X.isna().sum())  # check for nan

age                             0
sex                             0
education                       0
Employment_status               0
Personal_Net_Income_Category    0
living_arrangement              0
marital_status                  0
hours_on_social_media           0
loneliness_score                0
dtype: int64


In [20]:
print(y.isna().sum())
print(y.dtype)

1
float64


In [22]:

y.dropna(inplace=True)  #drop the entry with nan
X = X.loc[y.index]    # keep X and y aligned
print(y.isna().sum())

0


Nested CROSS-Validation

In [25]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold

# Sampler dictionary
samplers = {
    'none': None,
    'undersample': RandomUnderSampler(random_state=42),
    'smotetomek': SMOTETomek(random_state=42)
}

# Outer CV (voor consistentie)
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Hyperparameter grid
param_grid_lr = {
    'sampler': [samplers['none'], samplers['undersample'], samplers['smotetomek']],
    'classifier__C': [0.01, 0.1, 1, 10]
}

# Pipeline
pipeline_lr = ImbPipeline([
    ('sampler', SMOTETomek()),  # wordt geswitched op basis van de param_grid_lr
    ('scaler', StandardScaler()), # hier doen na datasplit anders dataleakage niet bij preprocessing.
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# GridSearch
grid_lr = GridSearchCV(pipeline_lr, param_grid=param_grid_lr, cv=3, scoring='f1', n_jobs=-1) # cv gaat automatisch stratifiedsplit
grid_lr.fit(X_train, y_train) # trainen van je gridsearch, hier ben je alleen de inner loop van de nested cv.
# om te checken of de 
# Evaluatie
print("Best parameters (LogReg):", grid_lr.best_params_)
y_pred = grid_lr.predict(X_test)
print(classification_report(y_test, y_pred))


NameError: name 'X_train' is not defined